In [1]:
#功能：股票数据的规整化处理
#思路：生成由以下列组成的数据表
    #TICKER_SYMBOL
    #year
    #month
    #CLOSE_PRICE
    #TURNOVER_VOL
    #TURNOVER_VALUE
    #MARKET_VALUE

![image.png](Images/MarketShare.png)

# 需要调用的函数

In [2]:
#功能：提取并增设两个列——year和month 
#思路：参见 addDataTimeFeatures的详解

def addDataTimeFeatures(df_s,date_column,DATETIME_UNITS):
     
    df_s['year'] = df_s[date_column].map(lambda x:x.year)
    df_st = pd.DataFrame({'MONTH':df_s[date_column].astype(str).apply(lambda x:x[5:7])})
    df_dict = pd.DataFrame({'MONTH':['01','02','03','04','05','06','07','08','09','10','11','12'],
                            'QUARTER':['03','03','03','06','06','06','09','09','09','12','12','12'],
                           'HALF':['06','06','06','06','06','06','12','12','12','12','12','12'],
                           'YEAR':['12','12','12','12','12','12','12','12','12','12','12','12']})
    df_output = pd.merge(df_st,df_dict,how='left')
    df_s = df_s.reset_index()
    df_s['month'] = df_output[DATETIME_UNITS]
    return(df_s)

# 参数设置

In [3]:
#文件路径
filename = 'Data/MarketData.xlsx' 

#统计周期的单位
DATETIME_UNITS="QUARTER"

#  读取数据

In [4]:
import pandas as pd     

df_shares = pd.DataFrame(pd.read_excel(filename,sheet_name=['DATA'],
                                       converters={'TICKER_SYMBOL':str,'PARTY_ID':str})['DATA'])

df_shares = df_shares[['TICKER_SYMBOL','END_DATE_','CLOSE_PRICE','TURNOVER_VOL','TURNOVER_VALUE','MARKET_VALUE']]

# 选择特征

In [5]:
df_shares.columns = ['TICKER_SYMBOL','END_DATE','CLOSE_PRICE','TURNOVER_VOL','TURNOVER_VALUE','MARKET_VALUE']

# 将'END_DATE'一列的类型改为日期类型

In [6]:
df_shares['END_DATE'] = pd.to_datetime(df_shares['END_DATE'])

# 增加时间特征列

In [7]:
 #按周期匹配出年月
df_shares = addDataTimeFeatures(df_shares,'END_DATE',DATETIME_UNITS)

# 分组统计

In [8]:
#按求和汇总
df_shares_g =  df_shares.groupby(['TICKER_SYMBOL','year','month'])\
    [['CLOSE_PRICE','TURNOVER_VOL','TURNOVER_VALUE','MARKET_VALUE']].\
    mean().\
    reset_index()
    

In [9]:
df_shares_g.head()

,TICKER_SYMBOL,year,month,CLOSE_PRICE,TURNOVER_VOL,TURNOVER_VALUE,MARKET_VALUE
0,000001,2007,03,4.475000,7.087334e+08,1.257592e+10,3.700954e+10
1,000001,2007,06,6.660667,5.369438e+08,1.455421e+10,5.458244e+10
2,000001,2007,09,9.856667,5.749509e+08,2.014285e+10,7.944289e+10
3,000001,2007,12,10.592000,3.911309e+08,1.565396e+10,8.802817e+10
4,000001,2008,03,8.168667,4.643084e+08,1.544389e+10,7.235700e+10
